In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load and Preprocess Datasets
def load_datasets():
    try:
        # File paths for the datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'

        # Load datasets into DataFrames
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)

        # Standardize column names and ensure lowercase for string content
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df

    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Helper Function: Calculate Production Projections
def calculate_production_projections(crop_data):
    # Filter for historical years (1997–2014)
    historical_data = crop_data[(crop_data['crop_year'] >= 1997) & (crop_data['crop_year'] <= 2014)]

    # Ensure production column exists
    if historical_data.empty or 'production' not in historical_data.columns:
        print("No sufficient historical data for production analysis.")
        return None

    # Extract unique crops to compute projections
    unique_crops = historical_data['crop'].str.lower().unique()
    projections = {}

    for crop in unique_crops:
        crop_specific_data = historical_data[historical_data['crop'].str.lower() == crop]
        if crop_specific_data.empty:
            continue

        # Group production by year
        yearly_production = crop_specific_data.groupby('crop_year')['production'].sum().reset_index()
        if len(yearly_production) < 2:
            print(f"Not enough data to compute projections for crop: {crop}")
            continue

        # Perform linear regression for future production projections
        X = yearly_production['crop_year'].values.reshape(-1, 1)
        y = yearly_production['production'].values
        model = LinearRegression()
        model.fit(X, y)

        # Project production for the next 5 years (2015–2019)
        future_years = np.arange(2015, 2020).reshape(-1, 1)
        future_production = model.predict(future_years)
        future_production = [max(0, value) for value in future_production]  # Ensure no negative values

        # Calculate year-over-year growth percentages
        annual_growth = []
        for i in range(len(future_production) - 1):
            growth_percentage = ((future_production[i + 1] - future_production[i]) / future_production[i]) * 100 if future_production[i] > 0 else 0
            annual_growth.append(max(0, growth_percentage))  # Ensure growth is non-negative

        projections[crop] = {
            "future_production": future_production,
            "annual_growth": annual_growth
        }

    return projections

# Step 2: Analyze a Single District with Exchange Balancing
def analyze_single_district(crops_df, nutrients_df, age_nutrients_df):
    district_name = input("Enter the district name: ").strip().lower()

    # Filter data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
    if selected_district_crops.empty:
        print(f"No crop data available for district '{district_name}'. Please check the input.")
        return None

    # Merge crop production data with nutrient composition data
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()

    # Calculate deficiencies for all age groups
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_percentage = (deficiency / age_rdi.values[0]) * 100  # In percentage
        deficiency_dict = {col: (def_value, def_percent) for col, def_value, def_percent in zip(nutrient_columns, deficiency, deficiency_percentage) if def_value > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify the group with the least deficiency
    least_deficient_group = min(deficiencies, key=lambda g: sum(def_val for def_val, _ in deficiencies[g].values()) if deficiencies[g] else float('inf'))
    if not deficiencies[least_deficient_group]:
        print(f"No deficiencies found for district '{district_name}'.")
        return None

    # Project future deficiencies (10 years)
    growth_rate = 0.05  # 5% annual growth rate
    future_deficiencies = {
        nutrient: (amount * (1 + growth_rate) ** 10, ((amount * (1 + growth_rate) ** 10 - amount) / amount) * 100)
        for nutrient, (amount, _) in deficiencies[least_deficient_group].items()
    }

    # Recommend crops for balancing deficiencies
    recommended_crops = {}
    for nutrient, (deficit, _) in deficiencies[least_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')
        recommended_crops[nutrient] = top_crops

    # Compute production projections for the district
    crop_projections = calculate_production_projections(selected_district_crops)

    # Display results
    print(f"\n=== Final Output for District: {district_name.title()} ===")
    print(f"Group with Least Deficiency: {least_deficient_group}")
    
    # Deficient Nutrients
    print("\nDeficient Nutrients (in % of RDI):")
    for nutrient, (amount, percent) in deficiencies[least_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f} ({percent:.2f}%)")
    
    # 10-Year Projected Deficiencies
    print("\n10-Year Projected Deficiencies (in % Growth from Now):")
    for nutrient, (amount, percent_growth) in future_deficiencies.items():
        print(f"- {nutrient}: {amount:.2f} ({percent_growth:.2f}%)")
    
    # Suggested Crops for Nutrient Balancing
    print("\nSuggested Crops for Balancing Nutrients:")
    for nutrient, crops in recommended_crops.items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    
    # 5-Year Production Projections
    print("\n5-Year Production Projections (with Annual Growth %):")
    if crop_projections:
        for crop, projection_data in crop_projections.items():
            print(f"- {crop.title()}:")
            print(f"  Future Production: {projection_data['future_production']}")
            print(f"  Annual Growth (%): {projection_data['annual_growth']}")
    else:
        print("No production projections available for the selected district.")

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Run district-level analysis dynamically
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    analyze_single_district(crops_df, nutrients_df, age_nutrients_df)


/tmp/ipykernel_481107/214773813.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_481107/214773813.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_481107/214773813.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!
Enter the district name: chitradurga
Not enough data to compute projections for crop: paddy
Not enough data to compute projections for crop: soyabean
Not enough data to compute projections for crop: arcanut (processed)
Not enough data to compute projections for crop: atcanut (raw)
Not enough data to compute projections for crop: cashewnut processed
Not enough data to compute projections for crop: cashewnut raw
Not enough data to compute projections for crop: papaya

=== Final Output for District: Chitradurga ===
Group with Least Deficiency: adult female crossbred (1000 number)

Deficient Nutrients (in % of RDI):
- carbohydrates: 109470305.00 (99.97%)
- protein: 16831825.70 (99.95%)
- fiber: 9745966.10 (99.96%)
- fat: 25540077.40 (99.96%)
- vitamin a: 255398766.00 (99.96%)
- vitamin c: 27370918.80 (99.99%)
- iron: 6567472.60 (99.96%)
- calcium: 364999873.80 (100.00%)
- potassium: 1715187275.00 (99.98%)
- magnesium: 113063391.00 (99.92%)

10

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load and Preprocess Datasets
def load_datasets():
    try:
        # File paths for the datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'

        # Load datasets into DataFrames
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)

        # Standardize column names and ensure lowercase for string content
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df

    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Helper Function: Calculate Production Projections
def calculate_production_projections(crop_data):
    # Filter for historical years (1997–2014)
    historical_data = crop_data[(crop_data['crop_year'] >= 1997) & (crop_data['crop_year'] <= 2014)]

    # Ensure production column exists
    if historical_data.empty or 'production' not in historical_data.columns:
        print("No sufficient historical data for production analysis.")
        return None

    # Extract unique crops to compute projections
    unique_crops = historical_data['crop'].str.lower().unique()
    projections = {}

    for crop in unique_crops:
        crop_specific_data = historical_data[historical_data['crop'].str.lower() == crop]
        if crop_specific_data.empty:
            continue

        # Group production by year
        yearly_production = crop_specific_data.groupby('crop_year')['production'].sum().reset_index()
        if len(yearly_production) < 2:
            print(f"Not enough data to compute projections for crop: {crop}")
            continue

        # Perform linear regression for future production projections
        X = yearly_production['crop_year'].values.reshape(-1, 1)
        y = yearly_production['production'].values
        model = LinearRegression()
        model.fit(X, y)

        # Project production for the next 5 years (2015–2019)
        future_years = np.arange(2015, 2020).reshape(-1, 1)
        future_production = model.predict(future_years)
        future_production = [max(0, value) for value in future_production]  # Ensure no negative values

        # Calculate year-over-year growth percentages
        annual_growth = []
        for i in range(len(future_production) - 1):
            growth_percentage = ((future_production[i + 1] - future_production[i]) / future_production[i]) * 100 if future_production[i] > 0 else 0
            annual_growth.append(max(0, growth_percentage))  # Ensure growth is non-negative

        projections[crop] = {
            "future_production": future_production,
            "annual_growth": annual_growth
        }

    return projections

# Step 2: Analyze a Single District with Exchange Balancing
def analyze_single_district(crops_df, nutrients_df, age_nutrients_df):
    district_name = input("Enter the district name: ").strip().lower()

    # Filter data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
    if selected_district_crops.empty:
        print(f"No crop data available for district '{district_name}'. Please check the input.")
        return None

    # Merge crop production data with nutrient composition data
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()

    # Calculate deficiencies for all age groups
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_percentage = (deficiency / age_rdi.values[0]) * 100  # In percentage
        deficiency_dict = {col: (def_value, def_percent) for col, def_value, def_percent in zip(nutrient_columns, deficiency, deficiency_percentage) if def_value > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify the group with the least deficiency
    least_deficient_group = min(deficiencies, key=lambda g: sum(def_val for def_val, _ in deficiencies[g].values()) if deficiencies[g] else float('inf'))
    if not deficiencies[least_deficient_group]:
        print(f"No deficiencies found for district '{district_name}'.")
        return None

    # Project future deficiencies (10 years)
    growth_rate = 0.05  # 5% annual growth rate
    future_deficiencies = {
        nutrient: (amount * (1 + growth_rate) ** 10, ((amount * (1 + growth_rate) ** 10 - amount) / amount) * 100)
        for nutrient, (amount, _) in deficiencies[least_deficient_group].items()
    }

    # Recommend crops for balancing deficiencies
    recommended_crops = {}
    for nutrient, (deficit, _) in deficiencies[least_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')
        recommended_crops[nutrient] = top_crops

    # Compute production projections for the district
    crop_projections = calculate_production_projections(selected_district_crops)

    # Find neighboring districts with nutrient surplus and include exchange logic
    surplus_nutrients = {}
    exchange_suggestions = {}
    neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)

    for nutrient in deficiencies[least_deficient_group]:
        surplus_nutrients[nutrient] = []
        for neighbor in neighboring_districts:
            neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
            neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
            neighbor_surplus = neighbor_data[nutrient].sum()

            # Add exchange logic: balance the nutrient deficit and surplus
            if neighbor_surplus > 0:
                exchange_amount = min(deficiencies[least_deficient_group][nutrient][0], neighbor_surplus)
                surplus_percentage = (neighbor_surplus / age_nutrients_df[nutrient].max()) * 100
                surplus_nutrients[nutrient].append({
                    'district': neighbor,
                    'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                    'surplus': neighbor_surplus,
                    'surplus_percentage': surplus_percentage,
                    'exchange_amount': exchange_amount
                })

                # Suggest exchange
                if nutrient not in exchange_suggestions:
                    exchange_suggestions[nutrient] = []
                exchange_suggestions[nutrient].append({
                    'receiving_district': district_name,
                    'donor_district': neighbor,
                    'deficit_fulfilled': exchange_amount,
                    'remaining_surplus': neighbor_surplus - exchange_amount,
                    'exchange_impact_percentage': (exchange_amount / neighbor_surplus) * 100
                })

     # Display results
    print(f"\n=== Final Output for District: {district_name.title()} ===")
    print(f"Group with Least Deficiency: {least_deficient_group}")

    # Deficient Nutrients
    print("\nDeficient Nutrients (in % of RDI):")
    for nutrient, (amount, percent) in deficiencies[least_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f} ({percent:.2f}%)")

    # 10-Year Projected Deficiencies
    print("\n10-Year Projected Deficiencies (in % Growth from Now):")
    for nutrient, (amount, percent_growth) in future_deficiencies.items():
        print(f"- {nutrient}: {amount:.2f} ({percent_growth:.2f}%)")

    # Suggested Crops for Nutrient Balancing
    print("\nSuggested Crops for Balancing Nutrients:")
    for nutrient, crops in recommended_crops.items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # 5-Year Production Projections
    print("\n5-Year Production Projections (with Annual Growth %):")
    if crop_projections:
        for crop, projection_data in crop_projections.items():
            print(f"- {crop.title()}:")
            print(f"  Future Production: {projection_data['future_production']}")
            print(f"  Annual Growth (%): {projection_data['annual_growth']}")
    else:
        print("No production projections available for the selected district.")

    # Neighboring Districts with Surplus and Exchange Impact
    print("\nNeighboring Districts with Surplus and Exchange Impact:")
    for nutrient, surplus in surplus_nutrients.items():
        print(f"- {nutrient}:")
        for district in surplus:
            print(f"  District: {district['district']}, Surplus: {district['surplus']} ({district['surplus_percentage']:.2f}%), Exchange Amount: {district['exchange_amount']}")

    # Exchange Policy for Nutrient Balance
    print("\nExchange Policy for Nutrient Balance (with % Impact):")
    for nutrient, exchanges in exchange_suggestions.items():
        print(f"- {nutrient}:")
        for exchange in exchanges:
            print(f"  Receiving District: {exchange['receiving_district']}")
            print(f"  Donor District: {exchange['donor_district']}")
            print(f"  Deficit Fulfilled: {exchange['deficit_fulfilled']} (Impact on Receiving District: {((exchange['deficit_fulfilled'] / deficiencies[least_deficient_group][nutrient][0]) * 100):.2f}%)")
            print(f"  Remaining Surplus in Donor District: {exchange['remaining_surplus']} (Impact on Donor District: {exchange['exchange_impact_percentage']:.2f}%)")


            
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Analyze a single district dynamically with exchange balancing and percentages
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    analyze_single_district(crops_df, nutrients_df, age_nutrients_df)
    
    

/tmp/ipykernel_481107/66946722.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_481107/66946722.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_481107/66946722.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!
Enter the district name: chitradurga
Not enough data to compute projections for crop: paddy
Not enough data to compute projections for crop: soyabean
Not enough data to compute projections for crop: arcanut (processed)
Not enough data to compute projections for crop: atcanut (raw)
Not enough data to compute projections for crop: cashewnut processed
Not enough data to compute projections for crop: cashewnut raw
Not enough data to compute projections for crop: papaya

=== Final Output for District: Chitradurga ===
Group with Least Deficiency: adult female crossbred (1000 number)

Deficient Nutrients (in % of RDI):
- carbohydrates: 109470305.00 (99.97%)
- protein: 16831825.70 (99.95%)
- fiber: 9745966.10 (99.96%)
- fat: 25540077.40 (99.96%)
- vitamin a: 255398766.00 (99.96%)
- vitamin c: 27370918.80 (99.99%)
- iron: 6567472.60 (99.96%)
- calcium: 364999873.80 (100.00%)
- potassium: 1715187275.00 (99.98%)
- magnesium: 113063391.00 (99.92%)

10